In [26]:
import xarray as xr
import numpy as np
import warnings
import matplotlib.pyplot as plt
import pandas as pd

warnings.filterwarnings("ignore")
plt.style.use("default")
from pathlib import Path
import cftime
import os, sys
import seaborn as sns
import cartopy
import cartopy.feature as cpf
from global_land_mask import globe
import CMIP6_light_map
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from shapely.geometry import box, mapping
import geopandas as gpd
import rioxarray
from matplotlib import cm
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import matplotlib.dates as mdates
import global_land_mask
from tqdm.notebook import trange, tqdm

sys.path.append("../CMIP6_downscale/")
from CMIP6_ridgeplot import CMIP6_ridgeplot
from xclim import ensembles
from CMIP6_IO import CMIP6_IO
from CMIP6_config import Config_albedo

In [27]:
def get_LME_records():
    lme_file = "gs://actea-shared/Shapefiles/LME66/LMEs66.shp"
    return gpd.read_file(lme_file)


def get_LME_records_plot():
    lme_file = "gs://actea-shared/Shapefiles/LME66_180/LME66_180.shp"
    return gpd.read_file(lme_file)


def create_colors(N):
    color = iter(cm.tab20b(np.linspace(0, 1, N)))
    return [next(color) for c in range(N)]


def add_LME_boundary(ax, LME, projection):
    shdf = get_LME_records_plot()
    shdf_sel = shdf[shdf["LME_NAME"] == LME]

    ax.add_geometries(shdf_sel["geometry"], projection, facecolor=None, edgecolor="k")
    return ax


def create_map(df, title, var_name, LME, period, anomalies=False, details=False):
    if LME in ["Barents Sea", "Arctic Ocean"]:
        projection = ccrs.NorthPolarStereo()  # ccrs.PlateCarree(central_longitude=0)
        extent = [-20, 90, 60, 90]
    else:
        projection = ccrs.PlateCarree(central_longitude=-180)
        extent = [-252, -100, 10, 65]
        extent = [-200, -145, 40, 80]
    res = "10m"
    ax = plt.figure(figsize=(16, 10)).gca(projection=projection)

    ax.coastlines(resolution=res, linewidth=0.6, color="black", alpha=0.8, zorder=4)
    ax.add_feature(cpf.BORDERS, linestyle=":", alpha=0.4)
    ax.add_feature(cpf.LAND, color="lightgrey")
    ax.set_extent(extent)

    xticks = np.linspace(extent[0], extent[1], 5)
    yticks = np.linspace(extent[2], extent[3], 5)

    ax.set_xticks(xticks, crs=ccrs.PlateCarree())
    ax.set_yticks(yticks, crs=ccrs.PlateCarree())
    lon_formatter = LongitudeFormatter(zero_direction_label=True)
    lat_formatter = LatitudeFormatter()
    ax.xaxis.set_major_formatter(lon_formatter)
    ax.yaxis.set_major_formatter(lat_formatter)

    # if var_name in ["par"]:
    # clb_label = "PAR ($W/m^{2}$)"
    ax = add_LME_boundary(ax, LME, projection)

    cs = ax.contourf(
        df["lon"],
        df["lat"],
        df[var_name],  # np.where(df["H"] < 0, df["H"], np.nan), # df[var_name],
        cmap=sns.color_palette("Spectral_r", as_cmap=True),
        transform=ccrs.PlateCarree(),
    )

    if title not in ["Bathymetry"]:
        clb = plt.colorbar(cs, shrink=0.5, extend="both")

    plt.xlabel("Longitude")
    plt.ylabel("Latitude")
    plt.savefig(
        f"Figures/{LME}_{var_name}_map.png",
        dpi=300,
        facecolor="w",
        transparent=False,
        bbox_inches="tight",
        pad_inches=0,
    )

    # if details:
    #    plt.savefig("Figures/Bottom_{}_july_sept_250m_zoomed_{}.png".format(var_name, period), dpi=300,
    #                facecolor='w',
    #                transparent=False,
    #                bbox_inches = 'tight',
    #                pad_inches = 0)
    # else:
    #    plt.savefig("Figures/Bottom_{}_july_sept_250m_{}.png".format(var_name, period), dpi=300,
    #            facecolor='w',
    #                transparent=False,
    #                bbox_inches = 'tight',
    #                pad_inches = 0)

    plt.show()


def get_data_within_LME(ds, var_name, LME, create_maps):
    # Extract the polygon defining the boundaries of the LME
    print(f"Working on LME: {LME}")

    shdf = get_LME_records()
    shdf_sel = shdf[shdf["LME_NAME"] == LME]

    # Rioxarray requires x and y dimensions - we convert these back to lon and lat later.
    # We also add the projection (lat-lon) so that rioxarray can do the clipping of the data according to the
    # shapefile.
    # ds = ds.rename({"lon": "x", "lat": "y"})
    ds = ds.rio.write_crs(4326)

    # Clip the data within the LME. We have to convert the polygon geometry to a geodataframe using
    # `shapely.geometry`. The clipping of data within the polygon is done using rioxarray.clip function
    clipped = ds.rio.clip(geometries=shdf_sel.geometry.apply(mapping), crs=ds.rio.crs)
    clipped = clipped.rename({"x": "lon", "y": "lat"})

    p1 = "2000-01-01 to 2020-01-01"

    create_maps = True
    if create_maps:
        string = f"{var_name} 2000-01-01 to 2020-01-01"
        clipped_p1 = (
            clipped.sel(time=slice("2000-01-01", "2020-01-01"))
            .mean({"time"})
            .sel(percentiles=50.0)
        )
        create_map(
            clipped_p1,
            string,
            var_name,
            LME,
            period=p1,
            anomalies=False,
            details=False,
        )
        plt.show()
    return clipped

In [28]:
def convert_to_180(ds):
    # We need to rename and reorganize the dimensions and names for spatial
    # coordinates to work with geopandas.
    ds = (ds.assign_coords(x=(((ds.x + 180) % 360) - 180))).sortby("x")
    ds = ds.assign_coords(
        lat=ds.lat[:, 0].values, lon=ds.lon[0, :].values, y=ds.lat[:, 0].values
    )
    return ds


def convert_time(ds):
    if not ds.indexes["time"].dtype in ["datetime64[ns]"]:
        time_objects = ds.indexes["time"].to_datetimeindex()
        ds = ds.assign_coords({"time": time_objects})
        ds = xr.decode_cf(ds)

    return ds


def get_area_averaged_ds(
    model,
    experiment_id,
    ds,
    var_name,
    LME,
    create_maps,
    outfile_name,
    frequency,
    start_time,
    end_time,
):
    # Frequency of smoothing the plots. This is the number of years to smooth over
    # if frequency is Annual.
    roll_frequency = 1

    ds = convert_to_180(ds)
    ds = ds.sel(time=slice(start_time, end_time))
    ds = convert_time(ds)
    if var_name in ["tas"]:
        ds = xr.where(ds > 100, ds - 273.15, ds)

    # Convert from kg/m-3 to mg/m-3
    if var_name in ["chl"]:
        print("chl.attrs", ds.attrs)
        #  ds[var_name]=ds[var_name]/1.0e6

    ds_lme = get_data_within_LME(ds, var_name, LME, create_maps)

    #  CMIP6_ridgeplot.ridgeplot(var_name,
    #                            None, outfile,
    #                            glorys=False, depth_threshold=None,
    #                            ds=ds_lme)

    ds = ds_lme.mean({"lat", "lon"}, skipna=True)

    if var_name in ["TOZ"]:
        ds = ds.assign(TOZ_std=ds_lme[var_name].std({"lat", "lon"}))

    df25 = ds[var_name].sel(percentiles=2.5).to_dataframe().dropna().reset_index()
    df500 = ds[var_name].sel(percentiles=50.0).to_dataframe().dropna().reset_index()
    df975 = ds[var_name].sel(percentiles=97.5).to_dataframe().dropna().reset_index()

    df = df500
    df["500"] = df500[var_name]
    df["25"] = df25[var_name]
    df["975"] = df975[var_name]

    df = df.reset_index()
    df = df.set_index("time")

    df = df.resample(frequency).mean()
    df["model_name"] = model
    df["LME"] = LME
    df["roll_mean"] = df["500"].rolling(roll_frequency).mean(skipna=True)
    df["roll_25"] = df["25"].rolling(roll_frequency).mean(skipna=True)
    df["roll_975"] = df["975"].rolling(roll_frequency).mean(skipna=True)

    df["model_scenario"] = experiment_id
    unique = "{}_{}".format(model, experiment_id)
    df["unique"] = unique

    model_info = {}
    model_info["model_name"] = model
    model_info["model_scenario"] = experiment_id
    model_info["model_var"] = var_name
    model_info["LME"] = LME

    return df, model_info

In [29]:
def calculate_velocity(df_uas: pd.DataFrame, df_vas: pd.DataFrame):
    print("calculate_velocity")

    u_ssp245 = df_uas["ssp245"]
    u_ssp585 = df_uas["ssp585"]

    v_ssp245 = df_vas["ssp245"]
    v_ssp585 = df_vas["ssp585"]

    df_ssp245 = u_ssp245
    df_ssp585 = u_ssp585

    df_ssp245["500"] = np.sqrt(
        np.power(u_ssp245["500"], 2) + np.power(v_ssp245["500"], 2)
    )
    df_ssp245["25"] = np.sqrt(np.power(u_ssp245["25"], 2) + np.power(v_ssp245["25"], 2))
    df_ssp245["975"] = np.sqrt(
        np.power(u_ssp245["975"], 2) + np.power(v_ssp245["975"], 2)
    )

    df_ssp585["500"] = np.sqrt(
        np.power(u_ssp585["500"], 2) + np.power(v_ssp585["500"], 2)
    )
    df_ssp585["25"] = np.sqrt(np.power(u_ssp585["25"], 2) + np.power(v_ssp585["25"], 2))
    df_ssp585["975"] = np.sqrt(
        np.power(u_ssp585["975"], 2) + np.power(v_ssp585["975"], 2)
    )

    df_ssp585 = df_ssp585.reset_index()
    df_ssp245 = df_ssp245.reset_index()

    return df_ssp245, df_ssp585


def create_timeseries_for_lme(
    varname: str, df: [pd.DataFrame], LME: str, stored_uas=None
):
    if stored_uas is None:
        df_ssp245 = df["ssp245"]
        df_ssp585 = df["ssp585"]
    else:
        varname = "velocity"
        df_ssp245, df_ssp585 = calculate_velocity(stored_uas, df)

    df_ssp245["date"] = df_ssp245.index
    df_ssp585["date"] = df_ssp585.index
    sns.set(style="white", rc={"axes.facecolor": (0, 0, 0, 0)}, font_scale=1.3)
    f = plt.figure(figsize=(16, 16))
    gs = f.add_gridspec(2, 1)
    ax = f.add_subplot(gs[0, 0])

    sns.set_palette(["#8172B3", "#64B5CD"])
    legend_on = True if varname == "tas" else False

    b = sns.lineplot(
        ax=ax,
        data=df_ssp245,
        x=df_ssp245["date"],
        y=df_ssp245["roll_mean"],
        hue=df_ssp245["model_scenario"],
        palette=["#8172B3"],
        alpha=0.95,
        ci=95,
        linewidth=5,
        legend=legend_on,
    )

    b = sns.lineplot(
        ax=ax,
        data=df_ssp585,
        x=df_ssp585["date"],
        y=df_ssp585["roll_mean"],
        hue=df_ssp585["model_scenario"],
        palette=["#64B5CD"],
        alpha=0.95,
        ci=95,
        linewidth=5,
        legend=legend_on,
    )

    cycle_colors = plt.rcParams["axes.prop_cycle"].by_key()["color"]
    ax.fill_between(
        df_ssp245["date"],
        df_ssp245["roll_25"],
        df_ssp245["roll_975"],
        color=cycle_colors[0],
        alpha=0.2,
    )
    ax.fill_between(
        df_ssp585["date"],
        df_ssp585["roll_25"],
        df_ssp585["roll_975"],
        color=cycle_colors[1],
        alpha=0.2,
    )

    b.tick_params(labelsize=30)
    b.set_xlabel("", fontsize=30)
    b.set_ylabel("", fontsize=30)

    if varname == "tas":
        plt.legend(loc="upper left", frameon=False, fontsize=32)

    ax.xaxis.set_major_locator(mdates.YearLocator(base=10))
    ax.xaxis.set_major_formatter(mdates.DateFormatter("%Y"))
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=-90)

    if not os.path.exists("Figures"):
        os.makedirs("Figures")
    plotfile = f"Figures/CMIP6_lightpaper_{varname}_{LME}.png"
    print(f"Created figure {plotfile}")
    plt.savefig(plotfile, dpi=300, bbox_inches="tight")
    plt.show()

In [30]:
%%time
experiment_ids = ["ssp245", "ssp585"]
frequency = "A"

# Create objects to reuse functionality to open files on Google Cloud storage
config = Config_albedo()
create_maps = False

period = "1979-01-01-2099-12-16"
start_time = "1979-01-01"
end_time = "2099-12-16"

source_ids = ["CMCC-ESM2", "CanESM5", "MPI-ESM1-2-LR", "UKESM1-0-LL", "MPI-ESM1-2-HR"]
source_ids_list = [
    ["r1i1p1f1"],
    [
        "r1i1p2f1",
        "r2i1p1f1",
        "r2i1p2f1",
        "r4i1p1f1",
        "r9i1p2f1",
        "r10i1p2f1",
        "r7i1p2f1",
        "r6i1p2f1",
        "r3i1p2f1",
        "r10i1p1f1",
    ],
    ["r10i1p1f2", "r1i1p1f1", "r2i1p1f1", "r4i1p1f2", "r5i1p1f1", "r6i1p1f2"],
    ["r1i1p1f2", "r2i1p1f2", "r3i1p1f2", "r4i1p1f2", "r8i1p2f1"],
    ["r1i1p1f1", "r2i1p1f1"],
]

ds_var_names = [
    "prw",
    "clt",
    "uas",
    "vas",
    "chl",
    "sithick",
    "siconc",
    "sisnthick",
    "sisnconc",
    "tas",
    "tos",
]
# s_var_names=["uas", "vas"]
# ds_var_names=["chl"]

LMES = ["Barents Sea", "Northern Bering - Chukchi Seas"]
toz_list = []
for LME in LMES:
    stored_uas = None
    for varname in ds_var_names:
        all_dfs = {}

        for experiment_id in experiment_ids:
            all_ds = []
            ens_stats = None

            for model_indx, source_id in enumerate(source_ids):
                member_ids = source_ids_list[model_indx]
                config.source_id = source_id
                config.variable_ids = [varname]
                model_counter = 0
                not_found = []
                io = CMIP6_IO()

                for config.member_id in member_ids:
                    io.organize_cmip6_netcdf_files_into_datasets(config, experiment_id)

                    for model in io.models:
                        if model.name == source_id:
                            if config.member_id in model.ds_sets.keys():
                                all_ds.append(model.ds_sets[config.member_id][varname])
                                model_counter += 1
                            else:
                                not_found.append(config.member_id)

                if model_counter == len(member_ids) - 1:
                    print(
                        f"Found correct number ({model_counter}) of member ids for model {model.name}"
                    )
                else:
                    print(
                        f"NOT correct number ({model_counter}) of member ids for model {model.name}"
                    )

            if len(all_ds) > 0:
                direc = config.cmip6_netcdf_dir
                ensemble_filename = io.format_netcdf_filename(
                    direc, "ensemble_percentiles", "", experiment_id, varname
                )
                ensemble_filename_stats = io.format_netcdf_filename(
                    direc, "ensemble", "", experiment_id, varname
                )

                ens_perc = io.open_dataset_on_gs(ensemble_filename)

                if not isinstance(ens_perc, xr.Dataset):
                    ens = ensembles.create_ensemble(
                        all_ds, multifile=False, backend="zarr", resample_freq="MS"
                    )

                    Path(ensemble_filename).parent.mkdir(parents=True, exist_ok=True)
                    Path(ensemble_filename_stats).parent.mkdir(
                        parents=True, exist_ok=True
                    )
                    ens_stats = ensembles.ensemble_mean_std_max_min(ens)
                    ens_perc = ensembles.ensemble_percentiles(
                        ens, values=[2.5, 50, 97.5], split=False
                    )
                    io.write_netcdf(ens_perc, ensemble_filename)
                    io.write_netcdf(ens_stats, ensemble_filename_stats)
                    io.upload_to_gcs(ensemble_filename)
                    io.upload_to_gcs(ensemble_filename_stats)

                outfile_name = ensemble_filename.replace(".nc", f"_{LME}.png")

                df, models_dict = get_area_averaged_ds(
                    "ensemble",
                    experiment_id,
                    ens_perc,
                    varname,
                    LME,
                    create_maps,
                    outfile_name,
                    frequency,
                    start_time,
                    end_time,
                )

                all_dfs[experiment_id] = df

        if varname == "uas":
            print("Storing uas", stored_uas)
            stored_uas = all_dfs.copy()

    create_timeseries_for_lme(varname, all_dfs, LME, stored_uas=stored_uas)

[CMIP6_IO] Opening file actea-shared/light/ssp245/CMCC-ESM2/CMIP6_CMCC-ESM2_r1i1p1f1_ssp245_prw.nc
NOT correct number (1) of member ids for model CMCC-ESM2
[CMIP6_IO] Opening file actea-shared/light/ssp245/CanESM5/CMIP6_CanESM5_r1i1p2f1_ssp245_prw.nc
[CMIP6_IO] Opening file actea-shared/light/ssp245/CanESM5/CMIP6_CanESM5_r2i1p1f1_ssp245_prw.nc
[CMIP6_IO] Opening file actea-shared/light/ssp245/CanESM5/CMIP6_CanESM5_r2i1p2f1_ssp245_prw.nc
[CMIP6_IO] Opening file actea-shared/light/ssp245/CanESM5/CMIP6_CanESM5_r4i1p1f1_ssp245_prw.nc
[CMIP6_IO] Opening file actea-shared/light/ssp245/CanESM5/CMIP6_CanESM5_r9i1p2f1_ssp245_prw.nc
[CMIP6_IO] Opening file actea-shared/light/ssp245/CanESM5/CMIP6_CanESM5_r10i1p2f1_ssp245_prw.nc
[CMIP6_IO] Opening file actea-shared/light/ssp245/CanESM5/CMIP6_CanESM5_r7i1p2f1_ssp245_prw.nc
[CMIP6_IO] Opening file actea-shared/light/ssp245/CanESM5/CMIP6_CanESM5_r3i1p2f1_ssp245_prw.nc
[CMIP6_IO] Opening file actea-shared/light/ssp245/CanESM5/CMIP6_CanESM5_r10i1p1f1_s

INFO:CMIP6-log:[CMIP6_IO] Finished uploading to : light/ssp245/ensemble_percentiles/CMIP6_ensemble_percentiles__ssp245_prw.nc
INFO:CMIP6-log:[CMIP6_IO] Finished uploading to : light/ssp245/ensemble/CMIP6_ensemble__ssp245_prw.nc


[CMIP6_IO] Opening file actea-shared/light/ssp585/CMCC-ESM2/CMIP6_CMCC-ESM2_r1i1p1f1_ssp585_prw.nc
NOT correct number (1) of member ids for model CMCC-ESM2
[CMIP6_IO] Opening file actea-shared/light/ssp585/CanESM5/CMIP6_CanESM5_r1i1p2f1_ssp585_prw.nc
[CMIP6_IO] Opening file actea-shared/light/ssp585/CanESM5/CMIP6_CanESM5_r2i1p1f1_ssp585_prw.nc
[CMIP6_IO] Opening file actea-shared/light/ssp585/CanESM5/CMIP6_CanESM5_r2i1p2f1_ssp585_prw.nc
[CMIP6_IO] Opening file actea-shared/light/ssp585/CanESM5/CMIP6_CanESM5_r4i1p1f1_ssp585_prw.nc
[CMIP6_IO] Opening file actea-shared/light/ssp585/CanESM5/CMIP6_CanESM5_r9i1p2f1_ssp585_prw.nc
[CMIP6_IO] Opening file actea-shared/light/ssp585/CanESM5/CMIP6_CanESM5_r10i1p2f1_ssp585_prw.nc
[CMIP6_IO] Opening file actea-shared/light/ssp585/CanESM5/CMIP6_CanESM5_r7i1p2f1_ssp585_prw.nc
[CMIP6_IO] Opening file actea-shared/light/ssp585/CanESM5/CMIP6_CanESM5_r3i1p2f1_ssp585_prw.nc
[CMIP6_IO] Opening file actea-shared/light/ssp585/CanESM5/CMIP6_CanESM5_r10i1p1f1_s

INFO:CMIP6-log:[CMIP6_IO] Finished uploading to : light/ssp585/ensemble_percentiles/CMIP6_ensemble_percentiles__ssp585_prw.nc
INFO:CMIP6-log:[CMIP6_IO] Finished uploading to : light/ssp585/ensemble/CMIP6_ensemble__ssp585_prw.nc


[CMIP6_IO] Opening file actea-shared/light/ssp245/CMCC-ESM2/CMIP6_CMCC-ESM2_r1i1p1f1_ssp245_clt.nc
NOT correct number (1) of member ids for model CMCC-ESM2
[CMIP6_IO] Opening file actea-shared/light/ssp245/CanESM5/CMIP6_CanESM5_r1i1p2f1_ssp245_clt.nc
[CMIP6_IO] Opening file actea-shared/light/ssp245/CanESM5/CMIP6_CanESM5_r2i1p1f1_ssp245_clt.nc
[CMIP6_IO] Opening file actea-shared/light/ssp245/CanESM5/CMIP6_CanESM5_r2i1p2f1_ssp245_clt.nc
[CMIP6_IO] Opening file actea-shared/light/ssp245/CanESM5/CMIP6_CanESM5_r4i1p1f1_ssp245_clt.nc
[CMIP6_IO] Opening file actea-shared/light/ssp245/CanESM5/CMIP6_CanESM5_r9i1p2f1_ssp245_clt.nc
[CMIP6_IO] Opening file actea-shared/light/ssp245/CanESM5/CMIP6_CanESM5_r10i1p2f1_ssp245_clt.nc
[CMIP6_IO] Opening file actea-shared/light/ssp245/CanESM5/CMIP6_CanESM5_r7i1p2f1_ssp245_clt.nc
[CMIP6_IO] Opening file actea-shared/light/ssp245/CanESM5/CMIP6_CanESM5_r3i1p2f1_ssp245_clt.nc
[CMIP6_IO] Opening file actea-shared/light/ssp245/CanESM5/CMIP6_CanESM5_r10i1p1f1_s

...

[CMIP6_IO] Opening file actea-shared/light/ssp245/CMCC-ESM2/CMIP6_CMCC-ESM2_r1i1p1f1_ssp245_tos.nc
NOT correct number (1) of member ids for model CMCC-ESM2
[CMIP6_IO] Opening file actea-shared/light/ssp245/CanESM5/CMIP6_CanESM5_r1i1p2f1_ssp245_tos.nc
[CMIP6_IO] Opening file actea-shared/light/ssp245/CanESM5/CMIP6_CanESM5_r2i1p1f1_ssp245_tos.nc
[CMIP6_IO] Opening file actea-shared/light/ssp245/CanESM5/CMIP6_CanESM5_r2i1p2f1_ssp245_tos.nc
[CMIP6_IO] Opening file actea-shared/light/ssp245/CanESM5/CMIP6_CanESM5_r4i1p1f1_ssp245_tos.nc
[CMIP6_IO] Opening file actea-shared/light/ssp245/CanESM5/CMIP6_CanESM5_r9i1p2f1_ssp245_tos.nc
[CMIP6_IO] Opening file actea-shared/light/ssp245/CanESM5/CMIP6_CanESM5_r10i1p2f1_ssp245_tos.nc
[CMIP6_IO] Opening file actea-shared/light/ssp245/CanESM5/CMIP6_CanESM5_r7i1p2f1_ssp245_tos.nc
[CMIP6_IO] Opening file actea-shared/light/ssp245/CanESM5/CMIP6_CanESM5_r3i1p2f1_ssp245_tos.nc
[CMIP6_IO] Opening file actea-shared/light/ssp245/CanESM5/CMIP6_CanESM5_r10i1p1f1_s

INFO:CMIP6-log:[CMIP6_IO] Finished uploading to : light/ssp245/ensemble_percentiles/CMIP6_ensemble_percentiles__ssp245_tos.nc
INFO:CMIP6-log:[CMIP6_IO] Finished uploading to : light/ssp245/ensemble/CMIP6_ensemble__ssp245_tos.nc


[CMIP6_IO] Opening file actea-shared/light/ssp585/CMCC-ESM2/CMIP6_CMCC-ESM2_r1i1p1f1_ssp585_tos.nc
NOT correct number (1) of member ids for model CMCC-ESM2
[CMIP6_IO] Opening file actea-shared/light/ssp585/CanESM5/CMIP6_CanESM5_r1i1p2f1_ssp585_tos.nc
[CMIP6_IO] Opening file actea-shared/light/ssp585/CanESM5/CMIP6_CanESM5_r2i1p1f1_ssp585_tos.nc
[CMIP6_IO] Opening file actea-shared/light/ssp585/CanESM5/CMIP6_CanESM5_r2i1p2f1_ssp585_tos.nc
[CMIP6_IO] Opening file actea-shared/light/ssp585/CanESM5/CMIP6_CanESM5_r4i1p1f1_ssp585_tos.nc
[CMIP6_IO] Opening file actea-shared/light/ssp585/CanESM5/CMIP6_CanESM5_r9i1p2f1_ssp585_tos.nc
[CMIP6_IO] Opening file actea-shared/light/ssp585/CanESM5/CMIP6_CanESM5_r10i1p2f1_ssp585_tos.nc
[CMIP6_IO] Opening file actea-shared/light/ssp585/CanESM5/CMIP6_CanESM5_r7i1p2f1_ssp585_tos.nc
[CMIP6_IO] Opening file actea-shared/light/ssp585/CanESM5/CMIP6_CanESM5_r3i1p2f1_ssp585_tos.nc
[CMIP6_IO] Opening file actea-shared/light/ssp585/CanESM5/CMIP6_CanESM5_r10i1p1f1_s

INFO:CMIP6-log:[CMIP6_IO] Finished uploading to : light/ssp585/ensemble_percentiles/CMIP6_ensemble_percentiles__ssp585_tos.nc
INFO:CMIP6-log:[CMIP6_IO] Finished uploading to : light/ssp585/ensemble/CMIP6_ensemble__ssp585_tos.nc


CPU times: user 35min 47s, sys: 3min, total: 38min 47s
Wall time: 1h 5min 34s


In [31]:
# Calculate the relative change in chlorophyll production between 1980-2000 and
# and 2080-2099 for each model scenario.
# df = pd.read_csv('test.csv', parse_dates=['time', 'date'])
df_ssp245 = df["ssp245"]
df_ssp585 = df["ssp585"]

df_ssp245["date"] = df_ssp245.index
df_ssp585["date"] = df_ssp585.index

clim245 = (df_ssp245["roll_mean"].loc["1980-01-01":"2000-01-01"]).mean(skipna=True)
clim585 = (df_ssp585["roll_mean"].loc["1980-01-01":"2000-01-01"]).mean(skipna=True)
std = (df_ssp245["roll_mean"].loc["2080-01-01":"2099-01-01"]).mean(skipna=True) + (
    df_ssp245["roll_std"].loc["2080-01-01":"2099-01-01"]
).mean(skipna=True)
std585 = (df_ssp585["roll_mean"].loc["2080-01-01":"2099-01-01"]).mean(skipna=True) + (
    df_ssp585["roll_std"].loc["2080-01-01":"2099-01-01"]
).mean(skipna=True)

df_ssp245["rel_change_std"] = ((std - float(clim245)) / float(clim245)) * 100.0
df_ssp245["rel_change"] = (
    (
        (df_ssp245["roll_mean"].loc["2080-01-01":"2099-01-01"]).mean(skipna=True)
        - float(clim245)
    )
    / float(clim245)
) * 100.0

df_ssp585["rel_change_std"] = ((std585 - float(clim245)) / float(clim245)) * 100.0
df_ssp585["rel_change"] = (
    (
        df_ssp585["roll_mean"].loc["2080-01-01":"2099-01-01"].mean(skipna=True)
        - float(clim245)
    )
    / float(clim245)
) * 100.0
print(df_ssp245[["time", "rel_change", "rel_change_std"]])
print(df_ssp585[["time", "rel_change", "rel_change_std"]])
sns.lineplot(y="roll_mean", x="time", data=df_ssp245)
sns.lineplot(y="roll_mean", x="time", data=df_ssp585)

NameError: name 'df' is not defined